In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader



import cv2
import numpy as np
import os



from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, ConcatDataset



import torch.nn as nn
import torchvision.models as models


from torchvision.utils import make_grid
import torch.nn.functional as F
import tqdm


import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader



import glob
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torchvision
import torchvision.transforms as transform
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, ConcatDataset
import torch.nn as nn
import torchvision.models as models
from torchvision.utils import make_grid
import torch.nn.functional as F
import tqdm

from sklearn.model_selection import train_test_split


Writeup

For the flower classification, I utilized the CNN model. All the resources used for this kaggle competition are linked as comments. 

After loading in the dataset using ImageFolder(), I wanted to try to prepare and augment the data, but since I got errors, I decided to transform and define the loader for testing and training data. 

I then created a CNN neural net class -- I took most of this code from online sources (look at the link provided in comments). Here the constructor is define and the data is supposed to pass through the layers. 

The model was then set to be trained and validated by processing the image in batches (i.e. for _, (_ ,_ ) in enumerate(loader)), reset the optimizer, pushed the data, receive the loss, and backpropagate.

After setting the model to an evaluation mode, I did the exact same thing. Then I applied this to the test set and output the predictions

In [ ]:
labels = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

print(labels)

train = transform.Compose([transform.Resize((100,100)),transform.ToTensor()])

print(train)


dtrain = ImageFolder('../input/ee379v-hw4/train', transform=train)

print(dtrain)

# https://pytorch.org/vision/stable/transforms.html

In [ ]:
dtrain, vtrain = train_test_split(dtrain, test_size=0.3, shuffle=True, random_state=42)

tloader = DataLoader(dtrain, batch_size= 20, shuffle=True,num_workers=2, pin_memory=True)

vloader = DataLoader(vtrain, batch_size=20, shuffle=False,num_workers=2, pin_memory=True)

#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
#using CNN

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.conv4 = nn.Conv2d(128, 256, 5)
        
        self.fc1 = nn.Linear(256, 10)
        
        self.pool = nn.MaxPool2d(2, 2)
        
    def forward(self, x):
        # print(x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        a,b,c,d = x.shape
        x = F.adaptive_avg_pool2d(x, 1).reshape(a, -1)
        x = self.fc1(x)
        return x
    
    
#https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

In [ ]:
import torch.optim as optim
from tqdm import tqdm
from time import sleep

model = Net()


# https://pytorch.org/docs/stable/optim.html
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# loss function
criterion = nn.CrossEntropyLoss()

In [ ]:
#credit goes to https://www.kaggle.com/code/alaaeddinebenzekri/brain-tumor-classification-with-pytorch/notebook
def train(model, trainloader, optimizer, criterion):
    model.train()

    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):
        counter += 1
        image, labels = data
        image = torch.tensor(image).to(device)
        labels = torch.tensor(labels).to(device)
        optimizer.zero_grad()

        outputs = model(image)

        loss = criterion(outputs, labels)
        train_running_loss += loss.item()

        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == labels).sum().item()

        loss.backward()

        optimizer.step()

    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc


def validate(model, testloader, criterion):
    model.eval()

    valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(testloader), total=len(testloader)):
            counter += 1
            
            image, labels = data
            image = torch.tensor(image).to(device)
            labels = torch.tensor(labels).to(device)

            outputs = model(image)

            loss = criterion(outputs, labels)
            valid_running_loss += loss.item()

            _, preds = torch.max(outputs.data, 1)
            valid_running_correct += (preds == labels).sum().item()
        

    epoch_loss = valid_running_loss / counter
    epoch_acc = 100. * (valid_running_correct / len(testloader.dataset))
    return epoch_loss, epoch_acc

In [ ]:
import time
for epoch in range(50):
    train(model, tloader ,optimizer, criterion)
    validate(model, vloader, criterion)
    time.sleep(5)

print('TRAINING COMPLETE')

In [ ]:
import copy
model_new = Net()

tedat = ImageFolder('../input/ee379v-hw4/', transform=train)
teload = DataLoader(tedat, batch_size = 20, num_workers=4, pin_memory=True)

In [ ]:
#https://discuss.pytorch.org/t/for-step-images-labels-in-enumerate-data-loader/86337
predictions = []

for step, (images, labels) in enumerate(teload):
  modeled = model_new(images)
  _, pred = torch.max(modeled.images, 1)
  predictions.append(pred)



In [ ]:
predictions_a = []
for a in predictions:
  for b in a:
    predictions_a.append(b)

In [ ]:
file_names = glob.glob("./input/ee379v-hw4/test/test/*")
file_names = [os.path.basename(x) for x in file_names]

In [ ]:
import pandas as pd
final = []

map_final = {
  0:0,
  1:1,
  2:2,
  3:3,
  4:4
}

In [ ]:
for prediction in predictions_a:
  final.append(labels[map_final[prediction]]) 

In [ ]:
submission= pd.DataFrame(final)

In [ ]:
submission["Id"] = file_names
submission["Category"] = final

In [ ]:
submission.to_csv('flower_preds.csv', index=False)